In [1]:
try:
    pass #실행할 코드
except:
    pass #에러가 났을 경우 코드

In [4]:
try:
    result = 123451/0 #여기서 zerodivision 에러가 나므로 except절로 이동하게 된다.
    print(f'연산의 결과는 {result}입니다.')
except:
    print('0으로 나눌 수 없습니다.')

0으로 나눌 수 없습니다.


In [10]:
%%time
try:
    x, y = input('숫자 두 개를 입력하세요.').split(' ')
except:
    print('값을 잘못 입력하셨습니다.')

숫자 두 개를 입력하세요. 12 0


CPU times: user 345 ms, sys: 115 ms, total: 460 ms
Wall time: 24.6 s


In [12]:
x = range(1,5)
try:
    for i in range(4):
        y = 10/x[i]
except ZeroDivisionError as e:
    print('0으로 나눌 수 없습니다.')
    print(e)
except IndexError as e:
    print('인덱스 에러 발생')
    print(e)
else:
    print('에러 없이 실행되었습니다.')

에러 없이 실행되었습니다.


In [15]:
try:
    x, y = map(int, (input("정수를 띄어쓰기로 구분하여 두 개 입력해 주세요.").split(' ')))
    result = x/y
except ValueError:
    print('값을 잘못 입력하셨습니다.')
except ZeroDivisionError:
    print('0으로 나눌 수 없습니다.')
else:
    print(result)

정수를 띄어쓰기로 구분하여 두 개 입력해 주세요. 10 20


0.5


In [16]:
try:
    raise KeyError
finally:
    print('무조건 실행됩니다.')

무조건 실행됩니다.


KeyError: 

In [18]:
try:
    x, y = map(int, (input("정수를 띄어쓰기로 구분하여 두 개 입력해 주세요.").split(' ')))
    result = x/y
except ValueError:
    print('값을 잘못 입력하셨습니다.')
except ZeroDivisionError:
    print('0으로 나눌 수 없습니다.')
else:
    print(result)
finally:
    print('프로그램을 종료합니다')

정수를 띄어쓰기로 구분하여 두 개 입력해 주세요. 10 0


0으로 나눌 수 없습니다.
프로그램을 종료합니다


In [22]:
import requests as rq #http 통신을 위한 requests import

response = rq.get('https://api.github.com/repositories', headers={'Accept':'application/vnd.github.v3+json'})

print(response.status_code) #status code 반환

200


In [30]:
#response에 대한 요소 확인
print(f'encoding: {response.encoding}')
print(f"content type : {response.headers['Content-Type']}")
print(f"server : {response.headers['server']}")

encoding: utf-8
content type : application/json; charset=utf-8
server : GitHub.com


In [34]:
import json
print(json.dumps(response.json()[0], indent=2)[:200])

{
  "id": 1,
  "node_id": "MDEwOlJlcG9zaXRvcnkx",
  "name": "grit",
  "full_name": "mojombo/grit",
  "private": false,
  "owner": {
    "login": "mojombo",
    "id": 1,
    "node_id": "MDQ6VXNlcjE=",



In [37]:
response = rq.get('https://api.github.com/search/repositories')
print(response.status_code)

422


In [61]:
response = rq.get('https://api.github.com/search/repositories',
                 params={'q': 'data_science+language:python'},
                 headers={'Accept':'application/vnd.github.v3.text_match+json'})
print(response.status_code)

200


In [62]:
r = response.json()

In [69]:
print([r['items'][i]['name'] for i in range(len(r['items']))])

['data-science-from-scratch', 'PySyft', 'data-science-blogs', 'galaxy', 'DataCamp', 'data-scientist-roadmap', 'dsp', 'cookiecutter-data-science', 'Kaggler', 'kedro', 'book_sample', 'PDA_Book', 'metaflow', 'data-science-template', 'intro_ds', 'lale', 'heamy', 'datajoint-python', 'python-data-science-workshop', 'd6tflow', 'nebari', 'data-science-ipython-notebooks', 'Building-Data-Science-Applications-with-FastAPI', 'Uber-DS-Challenge', 'the_data_science_handbook', 'dsb-2017', 'DSB2018', 'ufora', 'xam', 'exercises']


In [70]:
response = rq.get('https://api.github.com/repos/pytorch/pytorch/issues',
                 headers={'Accept':'application/vnd.github.v3.text-match+json'})
print('Response Code', response.status_code)
print('Number of Contents', len(response.json()))

Response Code 200
Number of Contents 30


In [75]:
import sys
sys.setrecursionlimit(10**5)

def get_all_pages(url, params=None, headers=None):
    output_json=[]
    response = rq.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json += get_all_pages(next_url, params, headers)
    return output_json

In [76]:
import pandas as pd

out = get_all_pages(
    "https://api.github.com/repos/pytorch/pytorch/issues/comments",
    params={
        'since': '2022-01-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept' : 'application/vnd.github.v3+json'}
)

df = pd.DataFrame(out)
print(df['body'].count())
df[['id', 'created_at','body']].sample(1)

240


,id,created_at,body
134,1445368927,2023-02-26T13:59:47Z,### Merge started\nYour change will be merged ...


In [78]:
response = rq.head('https://api.github.com/repos/pytorch/pytorch/issues/comments')
print('X-Ratelimit-Limit', response.headers['X-Ratelimit-Limit'])
print('X-Ratelimit-Remaining', response.headers['X-Ratelimit-Remaining'])

import datetime as dt
print('Rate limit reset at', dt.datetime.fromtimestamp(int(response.headers['X-RateLimit-Reset'])).strftime('%c'))

X-Ratelimit-Limit 60
X-Ratelimit-Remaining 58
Rate limit reset at Mon Feb 27 17:52:24 2023


In [82]:
from datetime import datetime as dt
import time

def handle_rate_limits(response):
    now = dt.now()
    reset_time = dt.fromtimestamp(int(response.headers['X-RateLimit-Reset']))
    remaining_requests = response.headers['X-Ratelimit-Remaining']
    remaining_time =(reset_time-now).total_seconds()
    intervals = remaining_time / (1.0 +int(remaining_requests))
    
    print('Sleeping for', intervals)
    time.sleep(intervals)
    return True

In [83]:
import sys
sys.setrecursionlimit(10**5)

def get_all_pages(url, params=None, headers=None):
    output_json=[]
    response = rq.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None and handle_rate_limits(response):
                output_json += get_all_pages(next_url, params, headers)
    return output_json

In [ ]:
import pandas as pd

out = get_all_pages(
    "https://api.github.com/repos/pytorch/pytorch/issues/comments",
    params={
        'since': '2022-01-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept' : 'application/vnd.github.v3+json'}
)

df = pd.DataFrame(out)
print(df['body'].count())
df[['id', 'created_at','body']].sample(1)

In [ ]:
#올인언 코드

from datetime import datetime as dt
import time
import sys
import pandas as pd

sys.setrecursionlimit(10**5)

def handle_rate_limits(response):
    now = dt.now()
    reset_time = dt.fromtimestamp(int(response.headers['X-RateLimit-Reset']))
    remaining_requests = response.headers['X-Ratelimit-Remaining']
    remaining_time =(reset_time-now).total_seconds()
    intervals = remaining_time / (1.0 +int(remaining_requests))
    
    print('Sleeping for', intervals)
    time.sleep(intervals)
    return True

def get_all_pages(url, params=None, headers=None):
    output_json=[]
    response = rq.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None and handle_rate_limits(response):
                output_json += get_all_pages(next_url, params, headers)
    return output_json

out = get_all_pages(
    "https://api.github.com/repos/pytorch/pytorch/issues/comments",
    params={
        'since': '2022-01-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept' : 'application/vnd.github.v3+json'}
)

df = pd.DataFrame(out)
print(df['body'].count())
df[['id', 'created_at','body']].sample(1)

In [92]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

retry_strategy = Retry(
    total = 5,
    status_forcelist=[500, 503, 504],
    backoff_factor=1
)

retry_adapter = HTTPAdapter(max_retries=retry_strategy)

http = requests.Session()
http.mount("https://", retry_adapter)

response = http.get('https://api.github.com/search/repositories', 
                   params = {'q': 'data_science+language:python'})

for item in response.json()['items'][:5]:
    print(item['name'])

data-science-from-scratch
PySyft
data-science-blogs
galaxy
DataCamp


In [ ]:
from datetime import datetime as dt
import time
import sys
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

sys.setrecursionlimit(10**5)

def handle_rate_limits(response):
    now = dt.now()
    reset_time = dt.fromtimestamp(int(response.headers['X-RateLimit-Reset']))
    remaining_requests = response.headers['X-Ratelimit-Remaining']
    remaining_time =(reset_time-now).total_seconds()
    intervals = remaining_time / (1.0 +int(remaining_requests))
    
    print('Sleeping for', int(intervals))
    time.sleep(intervals)
    return True

def get_all_pages(url, params=None, headers=None):
    output_json=[]
    retry_strategy = Retry(
        total = 5,
        status_forcelist=[500, 503, 504],
        backoff_factor=1
    )

    retry_adapter = HTTPAdapter(max_retries=retry_strategy)

    http = requests.Session()
    http.mount("https://", retry_adapter)
    
    response = http.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None and handle_rate_limits(response):
                output_json += get_all_pages(next_url, params, headers)
    return output_json

out = get_all_pages(
    "https://api.github.com/repos/pytorch/pytorch/issues/comments",
    params={
        'since': '2022-01-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept' : 'application/vnd.github.v3+json'}
)

df = pd.DataFrame(out)
print(df['body'].count())
df[['id', 'created_at','body']].sample(1)

In [96]:
odart = rq.get('https://opendart.fss.or.kr/api/list.json')
print('Response Code', response.status_code)

Response Code 200


In [97]:
print(odart.json())

{'message': '정의되지 않은 오류가 발생하였습니다.', 'status': '900'}


In [98]:
odart.headers

{'Connection': 'keep-alive', 'Set-Cookie': 'WMONID=5vgYaeq1fbv; Expires=Tue, 27-Feb-2024 17:22:55 GMT; Path=/', 'Date': 'Mon, 27 Feb 2023 08:22:55 GMT', 'Content-Type': 'application/json;charset=UTF-8', 'Transfer-Encoding': 'chunked'}